In [8]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import time
import psutil

# Function to get CPU utilization and memory consumption
def get_system_status():
    cpu_percent = psutil.cpu_percent()
    memory_percent = psutil.virtual_memory().percent
    return cpu_percent, memory_percent

# Node class for the Decision Tree
class Node:
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, value=None):
        self.feature_index = feature_index  # Index of feature to split on
        self.threshold = threshold          # Threshold value to split on
        self.left = left                    # Left subtree
        self.right = right                  # Right subtree
        self.value = value                  # Class label for leaf nodes

# Decision Tree class
class DecisionTree:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        self.root = None
    def _predict_tree(self, x, node):
     if node is None:
        return None  # Handle case where the node is None

     if node.value is not None:
        return node.value

     if x[node.feature_index] <= node.threshold:
        return self._predict_tree(x, node.left)
     else:
        return self._predict_tree(x, node.right)

    def fit(self, X, y):
        self.root = self._build_tree(X, y, depth=0)

    def _build_tree(self, X, y, depth):
        num_samples, num_features = X.shape
        unique_classes, counts = np.unique(y, return_counts=True)
        most_common_class = unique_classes[np.argmax(counts)]

        # Stopping criteria
        if depth == self.max_depth or len(np.unique(y)) == 1 or num_samples <= 1:
            return Node(value=most_common_class)

        # Find the best split
        best_split = self._find_best_split(X, y)

        if best_split is not None:
            feature_index, threshold = best_split
            left_mask = X[:, feature_index] <= threshold
            right_mask = ~left_mask

            left_subtree = self._build_tree(X[left_mask], y[left_mask], depth + 1)
            right_subtree = self._build_tree(X[right_mask], y[right_mask], depth + 1)

            return Node(feature_index=feature_index, threshold=threshold, left=left_subtree, right=right_subtree)

        return Node(value=most_common_class)

    def _find_best_split(self, X, y):
        num_samples, num_features = X.shape

        if num_samples <= 1:
            return None

        # Calculate the Gini impurity for the current node
        current_gini = self._gini_impurity(y)

        best_gini = 1
        best_split = None

        for feature_index in range(num_features):
            thresholds = np.unique(X[:, feature_index])
            for threshold in thresholds:
                left_mask = X[:, feature_index] <= threshold
                right_mask = ~left_mask

                if np.sum(left_mask) > 0 and np.sum(right_mask) > 0:
                    left_gini = self._gini_impurity(y[left_mask])
                    right_gini = self._gini_impurity(y[right_mask])

                    weighted_gini = (np.sum(left_mask) / num_samples) * left_gini + \
                                    (np.sum(right_mask) / num_samples) * right_gini

                    if weighted_gini < best_gini:
                        best_gini = weighted_gini
                        best_split = (feature_index, threshold)

        return best_split

    def _gini_impurity(self, y):
        _, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        gini = 1 - np.sum(probabilities**2)
        return gini

    def predict(self, X):
        return np.array([self._predict_tree(x, self.root) for x in X])



# Load MNIST data
mnist = fetch_openml('mnist_784', as_frame=True)
X, y = mnist.data.to_numpy().astype('float32'), mnist.target.astype('int')

# Normalize the pixel values to be in the range [0, 1]
X /= 255.0

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Decision Tree model
max_depth = 10
dt_model = DecisionTree(max_depth=max_depth)

# Monitoring overall system status
overall_cpu_utilization = []
overall_memory_utilization = []

# Training the Decision Tree model
start_time = time.time()
log_list = []

for epoch in range(1):
    # Monitor system status and log
    cpu_percent, memory_percent = get_system_status()
    overall_cpu_utilization.append(cpu_percent)
    overall_memory_utilization.append(memory_percent)
    log_list.append({'Epoch': epoch + 1, 'CPU Utilization': cpu_percent, 'Memory Usage (%)': memory_percent})

# Calculate the time taken for training
training_time = time.time() - start_time
print(f"Time taken for training: {training_time:.2f} seconds")

# Print the overall CPU and memory utilization
print(f"Overall CPU Utilization: {np.mean(overall_cpu_utilization)}%")
print(f"Overall Memory Usage: {np.mean(overall_memory_utilization)}%")


# Make predictions on the test set
dt_predictions = dt_model.predict(X_test)

# Handle None values by replacing them with a default value (replace with your logic)
dt_predictions = np.array([0 if pred is None else pred for pred in dt_predictions])

# Convert predictions to integers
dt_predictions = dt_predictions.astype(int)

# Evaluate accuracy
dt_accuracy = accuracy_score(y_test, dt_predictions)
print(f"Decision Tree Accuracy: {dt_accuracy}")




Time taken for training: 0.02 seconds
Overall CPU Utilization: 13.0%
Overall Memory Usage: 59.5%
Decision Tree Accuracy: 0.09592857142857143


In [9]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import time
import psutil

# Function to get CPU utilization and memory consumption
def get_system_status():
    cpu_percent = psutil.cpu_percent()
    memory_percent = psutil.virtual_memory().percent
    return cpu_percent, memory_percent

# Node class for the Decision Tree
class Node:
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, value=None):
        self.feature_index = feature_index  # Index of feature to split on
        self.threshold = threshold          # Threshold value to split on
        self.left = left                    # Left subtree
        self.right = right                  # Right subtree
        self.value = value                  # Class label for leaf nodes

# Decision Tree class
class DecisionTree:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        self.root = None
    def _predict_tree(self, x, node):
     if node is None:
        return None  # Handle case where the node is None

     if node.value is not None:
        return node.value

     if x[node.feature_index] <= node.threshold:
        return self._predict_tree(x, node.left)
     else:
        return self._predict_tree(x, node.right)

    def fit(self, X, y):
        self.root = self._build_tree(X, y, depth=0)

    def _build_tree(self, X, y, depth):
        num_samples, num_features = X.shape
        unique_classes, counts = np.unique(y, return_counts=True)
        most_common_class = unique_classes[np.argmax(counts)]

        # Stopping criteria
        if depth == self.max_depth or len(np.unique(y)) == 1 or num_samples <= 1:
            return Node(value=most_common_class)

        # Find the best split
        best_split = self._find_best_split(X, y)

        if best_split is not None:
            feature_index, threshold = best_split
            left_mask = X[:, feature_index] <= threshold
            right_mask = ~left_mask

            left_subtree = self._build_tree(X[left_mask], y[left_mask], depth + 1)
            right_subtree = self._build_tree(X[right_mask], y[right_mask], depth + 1)

            return Node(feature_index=feature_index, threshold=threshold, left=left_subtree, right=right_subtree)

        return Node(value=most_common_class)

    def _find_best_split(self, X, y):
        num_samples, num_features = X.shape

        if num_samples <= 1:
            return None

        # Calculate the Gini impurity for the current node
        current_gini = self._gini_impurity(y)

        best_gini = 1
        best_split = None

        for feature_index in range(num_features):
            thresholds = np.unique(X[:, feature_index])
            for threshold in thresholds:
                left_mask = X[:, feature_index] <= threshold
                right_mask = ~left_mask

                if np.sum(left_mask) > 0 and np.sum(right_mask) > 0:
                    left_gini = self._gini_impurity(y[left_mask])
                    right_gini = self._gini_impurity(y[right_mask])

                    weighted_gini = (np.sum(left_mask) / num_samples) * left_gini + \
                                    (np.sum(right_mask) / num_samples) * right_gini

                    if weighted_gini < best_gini:
                        best_gini = weighted_gini
                        best_split = (feature_index, threshold)

        return best_split

    def _gini_impurity(self, y):
        _, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        gini = 1 - np.sum(probabilities**2)
        return gini

    def predict(self, X):
        return np.array([self._predict_tree(x, self.root) for x in X])



# Load MNIST data
mnist = fetch_openml('mnist_784', as_frame=True)
X, y = mnist.data.to_numpy().astype('float32'), mnist.target.astype('int')

# Normalize the pixel values to be in the range [0, 1]
X /= 255.0

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Decision Tree model
max_depth = 10
dt_model = DecisionTree(max_depth=max_depth)

# Monitoring overall system status
overall_cpu_utilization = []
overall_memory_utilization = []

# Training the Decision Tree model
start_time = time.time()
log_list = []

for epoch in range(1):
    # Monitor system status and log
    cpu_percent, memory_percent = get_system_status()
    overall_cpu_utilization.append(cpu_percent)
    overall_memory_utilization.append(memory_percent)
    log_list.append({'Epoch': epoch + 1, 'CPU Utilization': cpu_percent, 'Memory Usage (%)': memory_percent})

# Calculate the time taken for training
training_time = time.time() - start_time
print(f"Time taken for training: {training_time:.2f} seconds")

# Print the overall CPU and memory utilization
print(f"Overall CPU Utilization: {np.mean(overall_cpu_utilization)}%")
print(f"Overall Memory Usage: {np.mean(overall_memory_utilization)}%")


# Make predictions on the test set
dt_predictions = dt_model.predict(X_test)

# Handle None values by replacing them with a default value (replace with your logic)
dt_predictions = np.array([0 if pred is None else pred for pred in dt_predictions])

# Convert predictions to integers
dt_predictions = dt_predictions.astype(int)

# Evaluate accuracy
dt_accuracy = accuracy_score(y_test, dt_predictions)
print(f"Decision Tree Accuracy: {dt_accuracy}")




Time taken for training: 0.02 seconds
Overall CPU Utilization: 9.9%
Overall Memory Usage: 58.1%
Decision Tree Accuracy: 0.09592857142857143


In [10]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import time
import psutil

# Function to get CPU utilization and memory consumption
def get_system_status():
    cpu_percent = psutil.cpu_percent()
    memory_percent = psutil.virtual_memory().percent
    return cpu_percent, memory_percent

# Node class for the Decision Tree
class Node:
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, value=None):
        self.feature_index = feature_index  # Index of feature to split on
        self.threshold = threshold          # Threshold value to split on
        self.left = left                    # Left subtree
        self.right = right                  # Right subtree
        self.value = value                  # Class label for leaf nodes

# Decision Tree class
class DecisionTree:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        self.root = None
    def _predict_tree(self, x, node):
     if node is None:
        return None  # Handle case where the node is None

     if node.value is not None:
        return node.value

     if x[node.feature_index] <= node.threshold:
        return self._predict_tree(x, node.left)
     else:
        return self._predict_tree(x, node.right)

    def fit(self, X, y):
        self.root = self._build_tree(X, y, depth=0)

    def _build_tree(self, X, y, depth):
        num_samples, num_features = X.shape
        unique_classes, counts = np.unique(y, return_counts=True)
        most_common_class = unique_classes[np.argmax(counts)]

        # Stopping criteria
        if depth == self.max_depth or len(np.unique(y)) == 1 or num_samples <= 1:
            return Node(value=most_common_class)

        # Find the best split
        best_split = self._find_best_split(X, y)

        if best_split is not None:
            feature_index, threshold = best_split
            left_mask = X[:, feature_index] <= threshold
            right_mask = ~left_mask

            left_subtree = self._build_tree(X[left_mask], y[left_mask], depth + 1)
            right_subtree = self._build_tree(X[right_mask], y[right_mask], depth + 1)

            return Node(feature_index=feature_index, threshold=threshold, left=left_subtree, right=right_subtree)

        return Node(value=most_common_class)

    def _find_best_split(self, X, y):
        num_samples, num_features = X.shape

        if num_samples <= 1:
            return None

        # Calculate the Gini impurity for the current node
        current_gini = self._gini_impurity(y)

        best_gini = 1
        best_split = None

        for feature_index in range(num_features):
            thresholds = np.unique(X[:, feature_index])
            for threshold in thresholds:
                left_mask = X[:, feature_index] <= threshold
                right_mask = ~left_mask

                if np.sum(left_mask) > 0 and np.sum(right_mask) > 0:
                    left_gini = self._gini_impurity(y[left_mask])
                    right_gini = self._gini_impurity(y[right_mask])

                    weighted_gini = (np.sum(left_mask) / num_samples) * left_gini + \
                                    (np.sum(right_mask) / num_samples) * right_gini

                    if weighted_gini < best_gini:
                        best_gini = weighted_gini
                        best_split = (feature_index, threshold)

        return best_split

    def _gini_impurity(self, y):
        _, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        gini = 1 - np.sum(probabilities**2)
        return gini

    def predict(self, X):
        return np.array([self._predict_tree(x, self.root) for x in X])



# Load MNIST data
mnist = fetch_openml('mnist_784', as_frame=True)
X, y = mnist.data.to_numpy().astype('float32'), mnist.target.astype('int')

# Normalize the pixel values to be in the range [0, 1]
X /= 255.0

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Decision Tree model
max_depth = 10
dt_model = DecisionTree(max_depth=max_depth)

# Monitoring overall system status
overall_cpu_utilization = []
overall_memory_utilization = []

# Training the Decision Tree model
start_time = time.time()
log_list = []

for epoch in range(1):
    # Monitor system status and log
    cpu_percent, memory_percent = get_system_status()
    overall_cpu_utilization.append(cpu_percent)
    overall_memory_utilization.append(memory_percent)
    log_list.append({'Epoch': epoch + 1, 'CPU Utilization': cpu_percent, 'Memory Usage (%)': memory_percent})

# Calculate the time taken for training
training_time = time.time() - start_time
print(f"Time taken for training: {training_time:.2f} seconds")

# Print the overall CPU and memory utilization
print(f"Overall CPU Utilization: {np.mean(overall_cpu_utilization)}%")
print(f"Overall Memory Usage: {np.mean(overall_memory_utilization)}%")


# Make predictions on the test set
dt_predictions = dt_model.predict(X_test)

# Handle None values by replacing them with a default value (replace with your logic)
dt_predictions = np.array([0 if pred is None else pred for pred in dt_predictions])

# Convert predictions to integers
dt_predictions = dt_predictions.astype(int)

# Evaluate accuracy
dt_accuracy = accuracy_score(y_test, dt_predictions)
print(f"Decision Tree Accuracy: {dt_accuracy}")




Time taken for training: 0.01 seconds
Overall CPU Utilization: 20.3%
Overall Memory Usage: 62.7%
Decision Tree Accuracy: 0.09592857142857143


In [11]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import time
import psutil

# Function to get CPU utilization and memory consumption
def get_system_status():
    cpu_percent = psutil.cpu_percent()
    memory_percent = psutil.virtual_memory().percent
    return cpu_percent, memory_percent

# Node class for the Decision Tree
class Node:
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, value=None):
        self.feature_index = feature_index  # Index of feature to split on
        self.threshold = threshold          # Threshold value to split on
        self.left = left                    # Left subtree
        self.right = right                  # Right subtree
        self.value = value                  # Class label for leaf nodes

# Decision Tree class
class DecisionTree:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        self.root = None
    def _predict_tree(self, x, node):
     if node is None:
        return None  # Handle case where the node is None

     if node.value is not None:
        return node.value

     if x[node.feature_index] <= node.threshold:
        return self._predict_tree(x, node.left)
     else:
        return self._predict_tree(x, node.right)

    def fit(self, X, y):
        self.root = self._build_tree(X, y, depth=0)

    def _build_tree(self, X, y, depth):
        num_samples, num_features = X.shape
        unique_classes, counts = np.unique(y, return_counts=True)
        most_common_class = unique_classes[np.argmax(counts)]

        # Stopping criteria
        if depth == self.max_depth or len(np.unique(y)) == 1 or num_samples <= 1:
            return Node(value=most_common_class)

        # Find the best split
        best_split = self._find_best_split(X, y)

        if best_split is not None:
            feature_index, threshold = best_split
            left_mask = X[:, feature_index] <= threshold
            right_mask = ~left_mask

            left_subtree = self._build_tree(X[left_mask], y[left_mask], depth + 1)
            right_subtree = self._build_tree(X[right_mask], y[right_mask], depth + 1)

            return Node(feature_index=feature_index, threshold=threshold, left=left_subtree, right=right_subtree)

        return Node(value=most_common_class)

    def _find_best_split(self, X, y):
        num_samples, num_features = X.shape

        if num_samples <= 1:
            return None

        # Calculate the Gini impurity for the current node
        current_gini = self._gini_impurity(y)

        best_gini = 1
        best_split = None

        for feature_index in range(num_features):
            thresholds = np.unique(X[:, feature_index])
            for threshold in thresholds:
                left_mask = X[:, feature_index] <= threshold
                right_mask = ~left_mask

                if np.sum(left_mask) > 0 and np.sum(right_mask) > 0:
                    left_gini = self._gini_impurity(y[left_mask])
                    right_gini = self._gini_impurity(y[right_mask])

                    weighted_gini = (np.sum(left_mask) / num_samples) * left_gini + \
                                    (np.sum(right_mask) / num_samples) * right_gini

                    if weighted_gini < best_gini:
                        best_gini = weighted_gini
                        best_split = (feature_index, threshold)

        return best_split

    def _gini_impurity(self, y):
        _, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        gini = 1 - np.sum(probabilities**2)
        return gini

    def predict(self, X):
        return np.array([self._predict_tree(x, self.root) for x in X])



# Load MNIST data
mnist = fetch_openml('mnist_784', as_frame=True)
X, y = mnist.data.to_numpy().astype('float32'), mnist.target.astype('int')

# Normalize the pixel values to be in the range [0, 1]
X /= 255.0

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Decision Tree model
max_depth = 10
dt_model = DecisionTree(max_depth=max_depth)

# Monitoring overall system status
overall_cpu_utilization = []
overall_memory_utilization = []

# Training the Decision Tree model
start_time = time.time()
log_list = []

for epoch in range(1):
    # Monitor system status and log
    cpu_percent, memory_percent = get_system_status()
    overall_cpu_utilization.append(cpu_percent)
    overall_memory_utilization.append(memory_percent)
    log_list.append({'Epoch': epoch + 1, 'CPU Utilization': cpu_percent, 'Memory Usage (%)': memory_percent})

# Calculate the time taken for training
training_time = time.time() - start_time
print(f"Time taken for training: {training_time:.2f} seconds")

# Print the overall CPU and memory utilization
print(f"Overall CPU Utilization: {np.mean(overall_cpu_utilization)}%")
print(f"Overall Memory Usage: {np.mean(overall_memory_utilization)}%")


# Make predictions on the test set
dt_predictions = dt_model.predict(X_test)

# Handle None values by replacing them with a default value (replace with your logic)
dt_predictions = np.array([0 if pred is None else pred for pred in dt_predictions])

# Convert predictions to integers
dt_predictions = dt_predictions.astype(int)

# Evaluate accuracy
dt_accuracy = accuracy_score(y_test, dt_predictions)
print(f"Decision Tree Accuracy: {dt_accuracy}")




Time taken for training: 0.01 seconds
Overall CPU Utilization: 17.7%
Overall Memory Usage: 60.3%
Decision Tree Accuracy: 0.09592857142857143


In [12]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import time
import psutil

# Function to get CPU utilization and memory consumption
def get_system_status():
    cpu_percent = psutil.cpu_percent()
    memory_percent = psutil.virtual_memory().percent
    return cpu_percent, memory_percent

# Node class for the Decision Tree
class Node:
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, value=None):
        self.feature_index = feature_index  # Index of feature to split on
        self.threshold = threshold          # Threshold value to split on
        self.left = left                    # Left subtree
        self.right = right                  # Right subtree
        self.value = value                  # Class label for leaf nodes

# Decision Tree class
class DecisionTree:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        self.root = None
    def _predict_tree(self, x, node):
     if node is None:
        return None  # Handle case where the node is None

     if node.value is not None:
        return node.value

     if x[node.feature_index] <= node.threshold:
        return self._predict_tree(x, node.left)
     else:
        return self._predict_tree(x, node.right)

    def fit(self, X, y):
        self.root = self._build_tree(X, y, depth=0)

    def _build_tree(self, X, y, depth):
        num_samples, num_features = X.shape
        unique_classes, counts = np.unique(y, return_counts=True)
        most_common_class = unique_classes[np.argmax(counts)]

        # Stopping criteria
        if depth == self.max_depth or len(np.unique(y)) == 1 or num_samples <= 1:
            return Node(value=most_common_class)

        # Find the best split
        best_split = self._find_best_split(X, y)

        if best_split is not None:
            feature_index, threshold = best_split
            left_mask = X[:, feature_index] <= threshold
            right_mask = ~left_mask

            left_subtree = self._build_tree(X[left_mask], y[left_mask], depth + 1)
            right_subtree = self._build_tree(X[right_mask], y[right_mask], depth + 1)

            return Node(feature_index=feature_index, threshold=threshold, left=left_subtree, right=right_subtree)

        return Node(value=most_common_class)

    def _find_best_split(self, X, y):
        num_samples, num_features = X.shape

        if num_samples <= 1:
            return None

        # Calculate the Gini impurity for the current node
        current_gini = self._gini_impurity(y)

        best_gini = 1
        best_split = None

        for feature_index in range(num_features):
            thresholds = np.unique(X[:, feature_index])
            for threshold in thresholds:
                left_mask = X[:, feature_index] <= threshold
                right_mask = ~left_mask

                if np.sum(left_mask) > 0 and np.sum(right_mask) > 0:
                    left_gini = self._gini_impurity(y[left_mask])
                    right_gini = self._gini_impurity(y[right_mask])

                    weighted_gini = (np.sum(left_mask) / num_samples) * left_gini + \
                                    (np.sum(right_mask) / num_samples) * right_gini

                    if weighted_gini < best_gini:
                        best_gini = weighted_gini
                        best_split = (feature_index, threshold)

        return best_split

    def _gini_impurity(self, y):
        _, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        gini = 1 - np.sum(probabilities**2)
        return gini

    def predict(self, X):
        return np.array([self._predict_tree(x, self.root) for x in X])



# Load MNIST data
mnist = fetch_openml('mnist_784', as_frame=True)
X, y = mnist.data.to_numpy().astype('float32'), mnist.target.astype('int')

# Normalize the pixel values to be in the range [0, 1]
X /= 255.0

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Decision Tree model
max_depth = 10
dt_model = DecisionTree(max_depth=max_depth)

# Monitoring overall system status
overall_cpu_utilization = []
overall_memory_utilization = []

# Training the Decision Tree model
start_time = time.time()
log_list = []

for epoch in range(1):
    # Monitor system status and log
    cpu_percent, memory_percent = get_system_status()
    overall_cpu_utilization.append(cpu_percent)
    overall_memory_utilization.append(memory_percent)
    log_list.append({'Epoch': epoch + 1, 'CPU Utilization': cpu_percent, 'Memory Usage (%)': memory_percent})

# Calculate the time taken for training
training_time = time.time() - start_time
print(f"Time taken for training: {training_time:.2f} seconds")

# Print the overall CPU and memory utilization
print(f"Overall CPU Utilization: {np.mean(overall_cpu_utilization)}%")
print(f"Overall Memory Usage: {np.mean(overall_memory_utilization)}%")


# Make predictions on the test set
dt_predictions = dt_model.predict(X_test)

# Handle None values by replacing them with a default value (replace with your logic)
dt_predictions = np.array([0 if pred is None else pred for pred in dt_predictions])

# Convert predictions to integers
dt_predictions = dt_predictions.astype(int)

# Evaluate accuracy
dt_accuracy = accuracy_score(y_test, dt_predictions)
print(f"Decision Tree Accuracy: {dt_accuracy}")




Time taken for training: 0.01 seconds
Overall CPU Utilization: 15.4%
Overall Memory Usage: 61.8%
Decision Tree Accuracy: 0.09592857142857143
